In [1]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

from inverted_index_colab import *


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ayals\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# These will already be installed in the testing environment so disregard the 
# amount of time (~1 minute) it takes to install. 
!pip install -q pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

You should consider upgrading via the 'c:\users\ayals\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\users\ayals\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.
'apt' is not recognized as an internal or external command,
operable program or batch file.
You should consider upgrading via the 'c:\users\ayals\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.
'wget' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
# Initializing spark context
# create a spark context and session
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'

conf = SparkConf().set("spark.ui.port", "4050")
conf.set("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.2-s_2.12")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\users\ayals\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-d7192e75e369>", line 8, in <module>
    sc = pyspark.SparkContext(conf=conf)
  File "c:\users\ayals\appdata\local\programs\python\python39\lib\site-packages\pyspark\context.py", line 195, in __init__
    SparkContext._ensure_initialized(self, gateway=gateway, conf=conf)
  File "c:\users\ayals\appdata\local\programs\python\python39\lib\site-packages\pyspark\context.py", line 417, in _ensure_initialized
    SparkContext._gateway = gateway or launch_gateway(conf)
  File "c:\users\ayals\appdata\local\programs\python\python39\lib\site-packages\pyspark\java_gateway.py", line 103, in launch_gateway
    time.sleep(0.1)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File

TypeError: object of type 'NoneType' has no len()

In [ ]:
def load_file():
    pkl_file = "part15_preprocessed.pkl"

    try:
        if os.environ["assignment_2_data"] is not None:
            pkl_file = Path(os.environ["assignment_2_data"])
    except:
        Exception("Problem with one of the variables")

    assert os.path.exists(pkl_file), 'You must upload this file.'
    with open(pkl_file, 'rb') as f:
        pages = pickle.load(f)
    return pages

In [ ]:
# pages = sc.parallelize(load_file())
# pages.collect()
from pathlib import Path 
import os

# try:
#     if os.environ["wikidata_preprocessed"] is not None:
#       path = os.environ["wikidata_preprocessed"]+"/wikidumps/*"
# except:
#       path = "wikidumps/*"

# parquetFile = spark.read.parquet(path)
# parquetFile.show()

# spark.read.parquet('part15_preprocessed.).show()

docs = sc.parallelize(load_file()).toDF().limit(1000).rdd

title_docs = docs.map(lambda r: (r[0],r[1]))
body_docs = docs.map(lambda r: (r[0],r[2]))
anchor_docs = docs.map(lambda r: (r[0],r[3]))
# spark